<a href="https://colab.research.google.com/github/mimrancomsats/ProgrammingforAI_FALL25/blob/main/Lab_14_Bert_HuggingFace_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [ ]:
temp_data = pd.read_csv("/content/drive/MyDrive/Programming for AI_SPRING-25/Labs/IMDB Dataset.csv")
df = temp_data.iloc[:10000]

#print(texts)

X = df['review']
Y = df['sentiment']

print(type(X))
X = X.tolist()
encoder = LabelEncoder()
Y = encoder.fit_transform(Y).tolist()

#print(type(encoder.fit_transform(Y)))
print(type(Y))

#print(X)
#print(Y)

<class 'pandas.core.series.Series'>
<class 'list'>


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, from_pt=True)  # <-- force load from PyTorch checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSeq

In [ ]:
# Split dataset into train, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(X, Y, test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

print(f"Size of X: {len(X)}")
print(f"Size of Y: {len(Y)}")

# Get the sizes of each split
train_texts_size = len(train_texts)
temp_texts_size = len(temp_texts)
train_labels_size = len(train_labels)
temp_labels_size = len(temp_labels)

# Print the sizes
print(f"Size of train_texts: {train_texts_size}")
print(f"Size of temp_texts: {temp_texts_size}")
print(f"Size of train_labels: {train_labels_size}")
print(f"Size of temp_labels: {temp_labels_size}")

# Get the sizes of each split
val_texts_size = len(val_texts)
test_texts_size = len(test_texts)
val_labels_size = len(val_labels)
test_labels_size = len(test_labels)

# Print the sizes
print(f"Size of val_texts: {val_texts_size}")
print(f"Size of teest_texts: {test_texts_size}")
print(f"Size of val_labels: {val_labels_size}")
print(f"Size of test_labels: {test_labels_size}")

Size of X: 10000
Size of Y: 10000
Size of train_texts: 7000
Size of temp_texts: 3000
Size of train_labels: 7000
Size of temp_labels: 3000
Size of val_texts: 1500
Size of teest_texts: 1500
Size of val_labels: 1500
Size of test_labels: 1500


In [ ]:
# Tokenize the Dataset
def tokenize_dataset(texts, labels, tokenizer, max_length=128):
    inputs = tokenizer(texts, max_length=max_length, padding="max_length", truncation=True, return_tensors="tf")
    #print(inputs)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    print(input_ids)
    print(attention_mask)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)  # Explicitly set dtype
    print(labels)
    return input_ids, attention_mask, labels

In [ ]:
# Tokenize datasets
train_input_ids, train_attention_mask, train_labels = tokenize_dataset(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_mask, val_labels = tokenize_dataset(val_texts, val_labels, tokenizer)
test_input_ids, test_attention_mask, test_labels = tokenize_dataset(test_texts, test_labels, tokenizer)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


tf.Tensor(
[[  101  2129  2204 ... 18908  2075   102]
 [  101  2067  1999 ... 16625  2108   102]
 [  101  1045  2481 ...  1999  2023   102]
 ...
 [  101  1045  2064 ...  2006  3909   102]
 [  101  2023  2537 ...     0     0     0]
 [  101  1045  2001 ...  1000  1037   102]], shape=(7000, 128), dtype=int32)
tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 1 1]], shape=(7000, 128), dtype=int32)
tf.Tensor([0 0 0 ... 0 1 0], shape=(7000,), dtype=int32)
tf.Tensor(
[[  101  1045  2066 ...  1037  2210   102]
 [  101  2023  3185 ...  1012  1037   102]
 [  101  1045  2001 ...  2126  1996   102]
 ...
 [  101  2023  2089 ...  4349  1010   102]
 [  101  1998  1045 ...  2009  1010   102]
 [  101  2521 15782 ...  5233  1012   102]], shape=(1500, 128), dtype=int32)
tf.Tensor(
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]], shape=(1500, 128), dtype=int32)

In [ ]:
def create_data_loader(input_ids, attention_mask, labels, batch_size=16):
    dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": attention_mask}, labels))
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [ ]:
train_data = create_data_loader(train_input_ids, train_attention_mask, train_labels)
val_data = create_data_loader(val_input_ids, val_attention_mask, val_labels)
test_data = create_data_loader(test_input_ids, test_attention_mask, test_labels)

In [ ]:
def train_model(model, train_data, val_data, epochs=3, learning_rate=5e-5):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.BinaryCrossentropy()  # Binary crossentropy for binary classification
    metric = tf.keras.metrics.BinaryAccuracy()  # Binary accuracy metric

    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    history = model.fit(train_data, validation_data=val_data, epochs=epochs)
    return history

In [ ]:
# Train the model
train_model(model, train_data, val_data)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/3
438/438 [==============================] - 5773s 13s/step - loss: 0.6629 - binary_accuracy: 0.6091 - val_loss: 0.6895 - val_binary_accuracy: 0.5173
Epoch 2/3
438/438 [==============================] - 5710s 13s/step - loss: 0.6407 - binary_accuracy: 0.6230 - val_loss: 0.4659 - val_binary_accuracy: 0.7833
Epoch 3/3
94/94 [==============================] - 386s 4s/step - loss: 0.6887 - binary_accuracy: 0.5573
Test Accuracy: 0.5573
